<a href="https://colab.research.google.com/github/xyfzkd/pytorch_start/blob/master/jupyter/FastPhotoStyle_NVIDIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FastPhotoStyle
This shows how to run the FastPhotoStyle code on the Google Colab platform. Please see the author's repository [here](https://github.com/NVIDIA/FastPhotoStyle). I take no credit in the original work, merely hacking away to get it to run on here. 

Note: This is still a work in progress, there are artifacts of previous attempts still within this document. I am trying to get a fresh instance so that it can be fully tested, but for the moment I thought it would be beneficial to see my steps.

Any comments/problems please put [here](https://github.com/NVIDIA/FastPhotoStyle/issues/22).

## Dependencies

cupy is the difficult one to get working here, it doesn't seem to be able to find a default CUDA install on the Colab instances. In order to solve this, I installed it myself. Please note that there are some downloads required and this code can't be run without them, the downloads require you to login into NVIDIA's website hence I haven't scripted it. 

### CUDA
Required for cupy.

In [29]:
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb

--2019-05-06 08:31:59--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.182.215
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.182.215|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?UOVA5cmLzauuoGY7TxrIbeq6xAjySuxygGoHb9PoQ4JgFQpz7PYvv02h-shTKkfxSBySfNu4N5p7Y2mr9YQ8YAd5Dpeeezrl7iJoaELns9cTo5ze1vF3KhmfnXl3ur9NSq86eN3CeBhrAkakvo_F8zjNxiTFITpKZQSInP8SrxMWcLAtK--HEfR2xUOttWUtqyEpOUQYdp46B135KhblyZcEqg [following]
--2019-05-06 08:31:59--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?UOVA5cmLzauuoGY7TxrIbeq6xAjySuxygGoHb9PoQ4JgFQpz7PYvv02h-shTKkfxSBySfNu4N5p7Y2mr9YQ8YAd5Dpee

In [30]:
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb

(Reading database ... 140929 files and directories currently installed.)
Preparing to unpack cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb ...
Unpacking cuda-repo-ubuntu1604-8-0-local-ga2 (8.0.61-1) over (8.0.61-1) ...
Setting up cuda-repo-ubuntu1604-8-0-local-ga2 (8.0.61-1) ...
Note: Check first if apt-key functionality is needed at all - it probably isn't!
OK


In [31]:
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub

OK


In [32]:
!apt update

Get:1 file:/var/cuda-repo-8-0-local-ga2  InRelease
Ign:1 file:/var/cuda-repo-8-0-local-ga2  InRelease
Get:2 file:/var/cuda-repo-8-0-local-ga2  Release [574 B]
Get:2 file:/var/cuda-repo-8-0-local-ga2  Release [574 B]
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86

In [33]:
!apt install cuda-8-0 -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
cuda-8-0 is already the newest version (8.0.61-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.


In [0]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'

In [35]:
!apt install gcc-5 g++-5 -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
g++-5 is already the newest version (5.5.0-12ubuntu1).
gcc-5 is already the newest version (5.5.0-12ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 59 not upgraded.


In [36]:
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc 
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++

ln: failed to create symbolic link '/usr/local/cuda/bin/gcc': File exists
ln: failed to create symbolic link '/usr/local/cuda/bin/g++': File exists


In [0]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'

### CUDNN
cupy also requires CUDNN. Get deb file from [here](https://developer.nvidia.com/cudnn). I used cuDNN v7.0.5 (Dec 5, 2017), for CUDA 8.0 which had filename cudnn-8.0-linux-x64-v7.tgz

In [41]:
!wget https://developer.nvidia.com/compute/machine-learning/cudnn/secure/v7.0.5/prod/8.0_20171129/cudnn-8.0-linux-x64-v7

--2019-05-06 09:02:19--  https://developer.nvidia.com/compute/machine-learning/cudnn/secure/v7.0.5/prod/8.0_20171129/cudnn-8.0-linux-x64-v7
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.182.215
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.182.215|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2019-05-06 09:02:19 ERROR 403: Forbidden.



In [38]:
!tar -xzvf cudnn-8.0-linux-x64-v7.tgz

tar (child): cudnn-8.0-linux-x64-v7.tgz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [39]:
!cp cuda/include/cudnn.h /usr/local/cuda/include
!cp cuda/lib64/libcudnn* /usr/local/cuda/lib64
!chmod a+r /usr/local/cuda/include/cudnn.h

cp: cannot stat 'cuda/include/cudnn.h': No such file or directory
cp: cannot stat 'cuda/lib64/libcudnn*': No such file or directory
chmod: cannot access '/usr/local/cuda/include/cudnn.h': No such file or directory


### NCCL

Required by cupy. Get deb file from [here](https://developer.nvidia.com/nccl). I used NCCL v2.1.4, for CUDA 8.0, Jan 18, 2018 which had filename nccl-repo-ubuntu1604-2.1.4-ga-cuda8.0_1-1_amd64.deb.

In [42]:
!wget https://developer.nvidia.com/compute/machine-learning/nccl/secure/v2.4/prod/nccl-repo-ubuntu1404-2.4.2-ga-cuda10.1_1-1_amd64.deb

--2019-05-06 09:06:29--  https://developer.nvidia.com/compute/machine-learning/nccl/secure/v2.4/prod/nccl-repo-ubuntu1404-2.4.2-ga-cuda10.1_1-1_amd64.deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.182.215
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.182.215|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2019-05-06 09:06:29 ERROR 403: Forbidden.



In [12]:
!dpkg -i nccl-repo-ubuntu1604-2.1.4-ga-cuda8.0_1-1_amd64.deb

dpkg: error: cannot access archive 'nccl-repo-ubuntu1604-2.1.4-ga-cuda8.0_1-1_amd64.deb': No such file or directory


In [13]:
!sudo apt update

Get:1 file:/var/cuda-repo-8-0-local-ga2  InRelease
Ign:1 file:/var/cuda-repo-8-0-local-ga2  InRelease
Get:2 file:/var/cuda-repo-8-0-local-ga2  Release [574 B]
Get:2 file:/var/cuda-repo-8-0-local-ga2  Release [574 B]
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:14 https://de

In [14]:
!sudo apt-get install libnccl2=2.1.4-1+cuda8.0 libnccl-dev=2.1.4-1+cuda8.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Version '2.1.4-1+cuda8.0' for 'libnccl2' was not found
E: Version '2.1.4-1+cuda8.0' for 'libnccl-dev' was not found


### CUPY

In [16]:
!pip install cupy 

### Torch

In [17]:
!pip install http://download.pytorch.org/whl/cu80/torch-0.3.1-cp27-cp27mu-linux_x86_64.whl 
!pip install torchvision 

     |████████████████████████████████| 496.9MB 1.1MB/s 
  Found existing installation: torch 1.1.0
    Uninstalling torch-1.1.0:
      Successfully uninstalled torch-1.1.0


### Misc

In [18]:
!pip install scikit-umfpack

     |████████████████████████████████| 9.5MB 8.9MB/s 


In [19]:
! apt-get install -y axel

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  axel
0 upgraded, 1 newly installed, 0 to remove and 59 not upgraded.
Need to get 56.1 kB of archives.
After this operation, 181 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 axel amd64 2.16.1-1build1 [56.1 kB]
Fetched 56.1 kB in 0s (655 kB/s)
Selecting previously unselected package axel.
(Reading database ... 140904 files and directories currently installed.)
Preparing to unpack .../axel_2.16.1-1build1_amd64.deb ...
Unpacking axel (2.16.1-1build1) ...
Setting up axel (2.16.1-1build1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [20]:
!pip install scikit-umfpack

## Project Setup

### Models
Code taken from [this](https://stackoverflow.com/questions/25010369/wget-curl-large-file-from-google-drive/39225039#39225039) stackoverflow post to download models.

In [0]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [0]:
file_id = '1ENgQm9TgabE1R99zhNf5q6meBvX6WFuq'
destination = './data.zip'
download_file_from_google_drive(file_id, destination)

### Code

In [23]:
! git clone https://github.com/NVIDIA/FastPhotoStyle

Cloning into 'FastPhotoStyle'...
remote: Enumerating objects: 337, done.
remote: Total 337 (delta 0), reused 0 (delta 0), pack-reused 337
Receiving objects: 100% (337/337), 50.73 MiB | 26.79 MiB/s, done.
Resolving deltas: 100% (193/193), done.


In [24]:
! unzip data.zip -d FastPhotoStyle

Archive:  data.zip
  inflating: FastPhotoStyle/models/feature_invertor_conv1_1_mask.t7  
  inflating: FastPhotoStyle/models/feature_invertor_conv2_1_mask.t7  
  inflating: FastPhotoStyle/models/feature_invertor_conv3_1_mask.t7  
  inflating: FastPhotoStyle/models/feature_invertor_conv4_1_mask.t7  
  inflating: FastPhotoStyle/models/feature_invertor_conv5_1_mask.t7  
  inflating: FastPhotoStyle/models/vgg_normalised_conv1_1_mask.t7  
  inflating: FastPhotoStyle/models/vgg_normalised_conv2_1_mask.t7  
  inflating: FastPhotoStyle/models/vgg_normalised_conv3_1_mask.t7  
  inflating: FastPhotoStyle/models/vgg_normalised_conv4_1_mask.t7  
  inflating: FastPhotoStyle/models/vgg_normalised_conv5_1_mask.t7  


## Test

### Experiment 1

In [0]:
! mkdir -p FastPhotoStyle/images && rm -fr FastPhotoStyle/images/* && mkdir -p FastPhotoStyle/results && rm -fr FastPhotoStyle/results/*

In [26]:
! axel -n 1 http://freebigpictures.com/wp-content/uploads/shady-forest.jpg --output=FastPhotoStyle/images/content1.png  
! axel -n 1 https://vignette.wikia.nocookie.net/strangerthings8338/images/e/e0/Wiki-background.jpeg/revision/latest?cb=20170522192233 --output=FastPhotoStyle/images/style1.png

Initializing download: http://freebigpictures.com/wp-content/uploads/shady-forest.jpg
File size: 1895125 bytes
Opening output file FastPhotoStyle/images/content1.png
Starting download

[  0%]  .......... .......... .......... .......... ..........  [ 115.7KB/s]
[  2%]  .......... .......... .......... .......... ..........  [ 184.6KB/s]
[  5%]  .......... .......... .......... .......... ..........  [ 273.5KB/s]
[  8%]  .......... .......... .......... .......... ..........  [ 362.4KB/s]
[ 10%]  .......... .......... .......... .......... ..........  [ 374.2KB/s]
[ 13%]  .......... .......... .......... .......... ..........  [ 448.4KB/s]
[ 16%]  .......... .......... .......... .......... ..........  [ 522.5KB/s]
[ 18%]  .......... .......... .......... .......... ..........  [ 511.8KB/s]
[ 21%]  .......... .......... .......... .......... ..........  [ 575.4KB/s]
[ 24%]  .......... .......... .......... .......... ..........  [ 639.1KB/s]
[ 27%]  .......... .......... .......... ....

In [44]:
! cd FastPhotoStyle/images && convert -resize 25% content1.png content1.png && convert -resize 50% style1.png style1.png

/bin/bash: convert: command not found


In [43]:
!pip install convert

  Stored in directory: /root/.cache/pip/wheels/ab/75/1b/0b09797001c280131e5e4ea8e51913298f0f4e627f6ed9dfc4
Successfully built convert


In [45]:
! cd FastPhotoStyle && python demo.py


Traceback (most recent call last):
  File "demo.py", line 9, in <module>
    import process_stylization
  File "/content/FastPhotoStyle/process_stylization.py", line 14, in <module>
    from smooth_filter import smooth_filter
  File "/content/FastPhotoStyle/smooth_filter.py", line 328, in <module>
    from pynvrtc.compiler import Program
ImportError: No module named pynvrtc.compiler


In [46]:
from IPython.display import Image
Image("FastPhotoStyle/results/example1.png")
